In [13]:
import sys
sys.path.append('../')
sys.path.append('../../')
from CMRx4DFlowMaskGeneration import *
from Utils import *
from einops import rearrange
import matplotlib.pyplot as plt


In [14]:
# Read one subject/case from a directory containing the corresponding .mat files
data_dir = '/mnt/nas/nas3/openData/rawdata/4dFlow/ChallengeData/Demo/TaskR1R2/TrainSet/Aorta/Center012/Philips_30T_Ingenia/P005/'

# load a pre-reconstructed image (assumed to be already reconstructed)
img_csllr = load_mat(data_dir + '/img_csllr.mat', key='img_csllr')
segmask    = load_mat(data_dir + '/segmask.mat',    key='segmask')
img_csllr = img_csllr[()]
segmask   = segmask[()]

Nv, Nt, SPE, PE, FE = img_csllr.shape


In [15]:
R = 10
img_csllr_masked = img_csllr * segmask[None, None]

# Calculate original size (uncompressed)
original_size = img_csllr_masked.nbytes
print(f"Original size: {original_size / 1024 / 1024:.2f} MB")

# Save masked image as COO sparse format
save_coo_npz(f"{data_dir}/img_ktGaussian{R}_CS.npz", img_csllr_masked)

# Print compressed file size
file_size = os.path.getsize(f"{data_dir}/img_ktGaussian{R}_CS.npz")
print(f"Compressed file size: {file_size / 1024 / 1024:.2f} MB")


Original size: 134.77 MB
Compressed file size: 5.69 MB


In [16]:
# Load COO representation and reconstruct dense array from compressed .npz file
img_csllr_masked_reloaded = load_coo_npz(f"{data_dir}/img_ktGaussian{R}_CS.npz", as_dense=True)

# Max absolute error between reloaded and original
print(f"Max abs error: {np.max(np.abs(img_csllr_masked_reloaded - img_csllr_masked))}")

Max abs error: 0.0
